In [1]:
import gym
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from collections import deque
import matplotlib.pyplot as plt

In [3]:
class Actor(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.fc = Dense(128, activation='relu')
        self.out= Dense(action_size, activation='sigmoid',
                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))

    def call(self, x):
        x      = self.fc(x)
        policy = self.out(x)
        return policy

class Critic(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.fc = Dense(128, activation='relu')
        self.out= Dense(action_size, activation='None',
                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))

    def call(self,x):
        x      = self.fc(x)
        q_value= self.out(x)
        return q_value

In [5]:
class DDPGAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper params for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        
        # Experience Replay
        self.batch_size = 64
        self.train_start = 1000
        self.memory = deque(maxlen=10000)
        
        self.actor         = Actor(self.state_size, self.action_size)
        self.critic        = Critic(self.state_size, self.action_size)
        self.target_actor  = Actor(self.state_size, self.action_size)
        self.target_critic = Critic(self.state_size, self.action_size)
        
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def get_action(self, state):
        policy, _ = self.actor(state)
        policy = np.array(policy[0])
        return np.random.choice(self.action_size, 1, p=policy)[0]
        
    def train_model(self):
        # Train from Experience Replay
        states      = np.array([sample[0][0] for sample in mini_batch])
        actions     = np.array([sample[1] for sample in mini_batch])
        rewards     = np.array([sample[2] for sample in mini_batch])
        next_states = np.array([sample[3][0] for sample in mini_batch])
        dones       = np.array([sample[4] for sample in mini_batch])
        
        critic_params = self.critic.trainable_variables
        with tf.GradientTape() as tape:
            # Target actor network
            target_actions = self.target_actor(next_states)
            # Target critic network
            critic_values  = self.target_critic([next_states, target_actions])
            targets = rewards + (1 - dones) * critic_values[0]
            critic_loss = tf.reduce_mean(tf.square(critic_values - targets))
            
        critic_grads = tape.gradient(critic_loss, critic_params)
        self.optimizer.apply_gradients(zip(critic_grads, critic_params))

        actor_params  = self.actor.trainable_variables
        with tf.GradientTape() as tape:
            # actor network
            policys = self.actor(states)
            # critic network
            critic_values  = self.critic([states, actions])
            # actor network
            actor_loss = -tf.reduce_mean(critic_values)
            
        actor_grads = tape.gradient(actor_loss, actor_params)
        self.optimizer.apply_gradients(zip(actor_grads, actor_params))
    
    def update_target(self,tau=1.0):
        tau = max(0.0, min(tau, 1.0))
        for (target_net, net) in zip(   self.actor.trainable_variables,
                                        self.target_actions.trainable_variables):
            target_net.assign(tau * target_net + (1.0 - tau) * net)
        

In [6]:
%matplotlib tk
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = DDPGAgent(state_size, action_size)

    scores, episodes, losses = [], [], []
    score_avg = 0
    
    end = False
    
    fig = plt.figure(1)
    fig.clf()
    
    num_episode = 2000
    for e in range(num_episode):
        done = False
        score = 0
        loss_list = []
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        
        while not done:
            env.render()

            action = agent.get_action(state)

            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            score += reward
            reward = 0.1 if not done or score == 500 else -1

            loss = agent.train_model(state, action, reward, next_state, done)
            loss_list.append(loss)

            state = next_state
            if done:
                
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | loss: {:.4f}'.format(e, score_avg, np.mean(loss_list)))

                scores.append(score_avg)
                episodes.append(e)
                losses.append(np.mean(loss_list))
                plt.subplot(211)
                plt.plot(episodes, scores, 'b')
                plt.xlabel('episode')
                plt.ylabel('average score')
                plt.title('cartpole DDPG')
                
                plt.subplot(212)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode')
                plt.ylabel('loss')
                
                plt.savefig('./save_model/cartpole_ddpg.png')

                if score_avg > 400:
                    agent.model.save_weights('./save_model/cartpole_ddpg', save_format='tf')
                    end = True
                    break
        if end == True:
            np.save('./save_model/cartpole_ddpg_epi',episodes)
            np.save('./save_model/cartpole_ddpg_score',scores)
            np.save('./save_model/cartpole_ddpg_loss',losses)
            env.close()
            print("End")
            break

ValueError: Unknown activation function: None

In [ ]:
    num_episode = 1000
    for e in range(num_episode,2*num_episode):
        done = False
        score = 0
        loss_list = []
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        
        while not done:
            env.render()

            action = agent.get_action(state)

            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            score += reward
            reward = 0.1 if not done or score == 500 else -1

            loss = agent.train_model(state, action, reward, next_state, done)
            loss_list.append(loss)

            state = next_state
            if done:
                
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | loss: {:.4f}'.format(e, score_avg, np.mean(loss_list)))

                scores.append(score_avg)
                episodes.append(e)
                losses.append(np.mean(loss_list))
                plt.subplot(211)
                plt.plot(episodes, scores, 'b')
                plt.xlabel('episode')
                plt.ylabel('average score')
                plt.title('cartpole A2C')
                
                plt.subplot(212)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode')
                plt.ylabel('loss')
                
                plt.savefig('./save_model/cartpole_a2c.png')

                if score_avg > 400:
                    agent.model.save_weights('./save_model/cartpole_a2c', save_format='tf')
                    end = True
                    break;
        if end == True:
            np.save('./save_model/cartpole_a2c_epi',episodes)
            np.save('./save_model/cartpole_a2c_score',scores)
            np.save('./save_model/cartpole_a2c_loss',losses)
            env.close()
            print("End")
            break;